# Camera and Image
https://docs.opencv.org/3.4/d6/d0f/group__dnn.html

## Face Detection 

In [4]:
%matplotlib inline

import cv2
import numpy as np

import time

#### Networks

In [5]:
facenet = cv2.dnn.readNet("face-detection-retail-0005.bin","face-detection-retail-0005.xml")
agenet = cv2.dnn.readNet("age-gender-recognition-retail-0013.bin","age-gender-recognition-retail-0013.xml")
emonet = cv2.dnn.readNet("emotions-recognition-retail-0003.bin","emotions-recognition-retail-0003.xml")

#### Video Stream

In [6]:
# Start Video Stream
label = ('Female', 'Male')
emotion_label = ('Neutral', 'Happy', 'Sad', 'Surprise', 'Anger')
font = cv2.FONT_HERSHEY_SIMPLEX
cap = cv2.VideoCapture(0)
fps = 25
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
writer = cv2.VideoWriter('Mask Face Detection.mp4', cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))

while cv2.waitKey(1) < 0:
    ret, frame = cap.read()
    

    if not ret:
        break
 
    try:
        start_time = time.time()
        
        blob = cv2.dnn.blobFromImage(frame)
        facenet.setInput(blob)
        out = facenet.forward()


        for detection in out.reshape(-1, 7):
            confidence = float(detection[2])
            xmin = int(detection[3] * frame.shape[1])
            ymin = int(detection[4] * frame.shape[0])
            xmax = int(detection[5] * frame.shape[1])
            ymax = int(detection[6] * frame.shape[0])

            if confidence > 0.5:
                #cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color=(0, 255, 0))
                face_crop = frame[ymin-10:ymax+10,xmin-10:xmax+10]


                agebolb = cv2.dnn.blobFromImage(face_crop,size = (62,62))
                agenet.setInput(agebolb)
                ageout =  agenet.forward(['age_conv3','prob'])

                age = round(ageout[0].reshape(-1)[0]*100)
                gender = label[np.argmax(ageout[1])]



                # EmotionNet
                emobolb = cv2.dnn.blobFromImage(face_crop,size = (64,64))
                emonet.setInput(emobolb)
                emoout =  emonet.forward()    
                # EmotionOutput
                emotion = emotion_label[np.argmax(emoout.reshape(-1))]

                

                cv2.putText(frame,text="Age:{:.1f} Gender:{}".format(age, gender),

                            org=(xmin,ymin-10), 


                            fontFace=font,fontScale= 0.5,color=(125,255,125),thickness=2,lineType=cv2.LINE_AA)

                cv2.putText(frame,text="Emotion Status: {}".format(emotion),

                            org=(xmin,ymin-30),  # X. Y

                            fontFace=font,fontScale= 0.5,color=(0,255,125),thickness=2,lineType=cv2.LINE_AA)



        
        end_time = time.time()
        fps = np.round(1/(end_time-start_time))
        
        cv2.putText(frame,text="Current FPS: {}".format(fps),

        org=(10,30),  # X. Y

        fontFace=font,fontScale= 0.5,color=(0,0,255),thickness=2,lineType=cv2.LINE_AA)
        writer.write(frame)
        cv2.imshow('OpenVINO face detection', frame)
        

    except:
        pass
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
        

    
# When everything done, realease the capture and destroy the windows
cap.release()
writer.release()
cv2.destroyAllWindows()